## Preprocessing

In [57]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [58]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"], axis=1)

In [59]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()
unique_values

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [60]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_counts = application_df["APPLICATION_TYPE"].value_counts()
application_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [61]:
# Cutoff 
cutoff_value = 1000

# Create a list of application types to be replaced
replaced_data = list(application_counts[application_counts < cutoff_value].index)

# Replace in dataframe
for cutoff in replaced_data:
    application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(cutoff,"Other")

# Check to make sure replacement was successful
application_df["APPLICATION_TYPE"].value_counts()

APPLICATION_TYPE
T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: count, dtype: int64

In [62]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
class_values = application_df["CLASSIFICATION"].value_counts()
class_values

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [63]:
# You may find it helpful to look at CLASSIFICATION value counts >1
# Get CLASSIFICATION value counts
class_values = application_df["CLASSIFICATION"].value_counts()

# Filter CLASSIFICATION counts > 1
class_over_one = class_values[class_values > 1]

# Print the CLASSIFICATION counts > 1
class_over_one.head(6)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Name: count, dtype: int64

In [64]:
# Choose a cutoff value and create a list of classifications to be replaced
class_cutoff = 1000

# Create a list of classifications to be replaced
other_class = list(class_values[ class_values < class_cutoff].index)

# Replace in dataframe
for cutoff_values in other_class:
    application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(cutoff_values,"Other")

# Check to make sure replacement was successful
application_df["CLASSIFICATION"].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [65]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,True,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,True,False,True,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,True,False,False,False,True,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [66]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
x = application_df.drop(["IS_SUCCESSFUL"], axis=1).values

# Split the preprocessed data into a training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=78)

In [67]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
x_scaler = scaler.fit(x_train)

# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

## Compile, Train and Evaluate the Model

In [68]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
features_num = len(x_train[0])
node1 =  80
node2 = 30

neural_net = tf.keras.models.Sequential()

# First hidden layer
neural_net.add(tf.keras.layers.Dense(units=node1, input_dim=features_num, activation="relu"))

# Second hidden layer
neural_net.add(tf.keras.layers.Dense(units=node2, activation="relu"))

# Output layer
neural_net.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
neural_net.summary()

/Users/jasleenbrar/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 80)             │         3,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,741 (22.43 KB)

 Trainable params: 5,741 (22.43 KB)

 Non-trainable params: 0 (0.00 B)

In [69]:
# Compile the model
neural_net.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [70]:
# Train the model
trained_model = neural_net.fit(x_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 245us/step - accuracy: 0.6967 - loss: 0.6016
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 237us/step - accuracy: 0.7272 - loss: 0.5615
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 250us/step - accuracy: 0.7376 - loss: 0.5528
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 260us/step - accuracy: 0.7318 - loss: 0.5556
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 248us/step - accuracy: 0.7298 - loss: 0.5555
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 233us/step - accuracy: 0.7314 - loss: 0.5540
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 231us/step - accuracy: 0.7313 - loss: 0.5520
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 231us/step - accuracy: 0.7298 - loss: 0.5512
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 231us/step - accuracy: 0.7304 - loss: 0.5530
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 231us/step - accuracy: 0.7326 - loss: 0.5494
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 232us/step - accuracy: 0.7330 - loss: 0.5488
Epoch 12/100
804/80

In [71]:
print(f"x_test_scaled shape: {x_test_scaled.shape}")
print(f"y_test shape: {y_test.shape}")

x_test_scaled shape: (8575, 40)
y_test shape: (8575,)


In [72]:
# Evaluate the model using the test data
test_loss, model_accuracy = neural_net.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {test_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 315us/step - accuracy: 0.7248 - loss: 0.5601
Loss: 0.560089647769928, Accuracy: 0.724781334400177


In [74]:
# Export our model to HDF5 file
neural_net.save("HDF5.keras")